This is a generalization from [here](https://pennylane.ai/qml/demos/tutorial_variational_classifier/), applied to downsampled 4x4 MNIST Digits.

In [ ]:
!pip install pennylane --quiet

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
np.random.seed(0)

In [ ]:
import tensorflow as tf

import collections

"""
Module for MNIST Digits Dataset preprocessing.
https://www.tensorflow.org/quantum/tutorials/mnist

Python 3.10.11
"""

def filter_by_classes(x, y, classes=[3,6]):
    """
    Function that filters the MNIST Digits Dataset and returns samples on 'classes'.
    Parameters:
        x: Sample images.
        y: Sample labels.
        classes: List of classes to filter.
    Returns:
        x: x filtered by 'classes'.
        y: x filtered by 'classes'.
    """
    if not all(np.isin(classes, range(0, 10))):
        return ValueError("Classes must be a list of digits (0-9).")
    x, y = x[np.isin(y, classes)], y[np.isin(y, classes)]
    if len(classes)==2:
        return x, y==classes[-1]
    else:
        return x, y

def remove_contradicting(xs, ys):

    mapping = collections.defaultdict(set)
    orig_x = {}
    # Determine the set of labels for each unique image:
    for x,y in zip(xs,ys):
       orig_x[tuple(x.flatten())] = x
       mapping[tuple(x.flatten())].add(y)

    new_x = []
    new_y = []
    for flatten_x in mapping:
      x = orig_x[flatten_x]
      labels = mapping[flatten_x]
      if len(labels) == 1:
          new_x.append(x)
          new_y.append(next(iter(labels)))
      else:
          # Throw out images that match more than one label.
          pass

    num_uniq_3 = sum(1 for value in mapping.values() if len(value) == 1 and True in value)
    num_uniq_6 = sum(1 for value in mapping.values() if len(value) == 1 and False in value)
    num_uniq_both = sum(1 for value in mapping.values() if len(value) == 2)

    print("Number of unique images:", len(mapping.values()))
    print("Number of unique 3s: ", num_uniq_3)
    print("Number of unique 6s: ", num_uniq_6)
    print("Number of unique contradicting labels (both 3 and 6): ", num_uniq_both)
    print()
    print("Initial number of images: ", len(xs))
    print("Remaining non-contradicting unique images: ", len(new_x))

    return np.array(new_x), np.array(new_y)

def preprocess_mnist_digits(classes=[3,6]):
    """"
    Function that downloads the MNIST Digits dataset with TensorFlow and performs the following tasks:
        1. Normalizes pixel values from (0, 255) to (0, 1).
        2. By default, returns only 2 classes of digits for classification (this can be deactivated or modified by the 'classes' parameter).
        3. Resizes samples to 4x4 images.
        4. Removes samples that belong to multiple classes simultaneously.
        5. Converts images to binary."
    Parameters:
    Returns:
    """

    # Download dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Rescale the images from [0,255] to the [0.0,1.0] range.
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

    # Filter to get only '3's and '6's
    x_train, y_train = filter_by_classes(x_train, y_train, classes=classes)
    x_test, y_test = filter_by_classes(x_test, y_test, classes=classes)

    print("Number of filtered training examples:", len(x_train))
    print("Number of filtered test examples:", len(x_test))

    # Resize images to 4x4
    x_train_small = tf.image.resize(x_train, (4,4)).numpy()
    x_test_small = tf.image.resize(x_test, (4,4)).numpy()

    x_train_nocon, y_train_nocon = remove_contradicting(x_train_small, y_train)

    THRESHOLD = 0.5

    # Converts non contradicting samples to binary via threshold and converting bool to float.
    x_train_bin = np.array(x_train_nocon > THRESHOLD, dtype=np.float32)
    x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

    return x_train_bin.reshape(-1, 16), y_train_nocon, x_test_bin.reshape(-1, 16), y_test



In [ ]:
import requests

In [ ]:
response = requests.get(
    "https://raw.githubusercontent.com/XanaduAI/qml/master/demonstrations/variational_classifier/data/parity.txt",
    params={"downloadformat": "txt"})

In [ ]:
# data = np.array([int(char_) for vector in response.content.decode().split("\n")[:-1] for char_ in vector.split(" ")], requires_grad=False).reshape(-1, 5)
# X = data[:, :-1]
# Y = data[:, -1]
# Y = Y * 2 - np.ones(len(Y))  # shift label from {0, 1} to {-1, 1}

In [ ]:
X_train, Y_train, X_test, Y_test = preprocess_mnist_digits()
Y_train = np.where(Y_train==False, -1, 1)
Y_test = np.where(Y_test==False, -1, 1)

Number of filtered training examples: 12049
Number of filtered test examples: 1968
Number of unique images: 10387
Number of unique 3s:  5426
Number of unique 6s:  4912
Number of unique contradicting labels (both 3 and 6):  49

Initial number of images:  12049
Remaining non-contradicting unique images:  10338


In [ ]:
X_train

tensor([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32, requires_grad=True)

In [ ]:
def layer(w):
    for j in range(16):
        qml.Rot(w[j, 0], w[j, 1], w[j, 2], wires=j)

    for k in range(16):
        qml.CNOT(wires=[k, (k+1)%16])

    # qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    # qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    # qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    # qml.CNOT(wires=[0, 1])
    # qml.CNOT(wires=[1, 2])
    # qml.CNOT(wires=[2, 3])
    # qml.CNOT(wires=[3, 0])

In [ ]:
[0, 1] 0.23456

0.22234
22234 -> 2**3 3**4 ...


In [ ]:
dev = qml.device("default.qubit", wires=16)

@qml.qnode(dev, interface="autograd")
def circuit(weights, x):

    qml.BasisState(x, wires=range(16))

    for w in weights:
        layer(w)

    return qml.expval(qml.PauliZ(0))


In [ ]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [ ]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [ ]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [ ]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [ ]:
np.random.seed(0)
num_qubits = 16
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)


In [ ]:
weights_init[0].shape

(16, 3)

In [ ]:
opt = qml.NesterovMomentumOptimizer(0.5)
batch_size = 5

In [ ]:
X_train.requires_grad = False
Y_train.requires_grad = False

In [ ]:
weights = weights_init
bias = bias_init
for it in range(800):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    #predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    #acc = accuracy(Y_train, predictions)
    if it%50 == 0:
      print(
          "Iter: {:5d} | Bias: {:0.7f} ".format(
              it + 1, bias
          )
      )

Iter:     1 | Bias: -0.3997756 
Iter:    51 | Bias: 0.6073953 
Iter:   101 | Bias: 0.6001665 
Iter:   151 | Bias: -0.2001170 
Iter:   201 | Bias: -0.2002143 
Iter:   251 | Bias: -0.5562595 
Iter:   301 | Bias: -0.2004881 
Iter:   351 | Bias: 0.5999379 
Iter:   401 | Bias: 0.2036899 
Iter:   451 | Bias: 0.1999919 
Iter:   501 | Bias: 0.9999869 
Iter:   551 | Bias: -0.2000812 
Iter:   601 | Bias: -0.5995388 
Iter:   651 | Bias: 0.6002661 
Iter:   701 | Bias: 0.5967054 
Iter:   751 | Bias: -0.6013463 


In [ ]:
np.save("weights.npy", weights)

In [ ]:
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

In [ ]:
accuracy(Y_test, predictions)

0.5132113821138211

In [ ]:
square_loss(Y_test, predictions)

tensor(1.94715447, requires_grad=True)

In [ ]:
predictions

[tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., requires_grad=True),
 tensor(-1., r